# Table of Contents
1. [Things to Do](#Things-to-Do)
1. [Introduction](#Introduction)
1. [Set up Python](#Set-up-Python)
2. [Functions](#Functions)
3. [Getting Data](#Get-Data)
4. [Analyze Data](#Analyze-Data):
    - [Survival and Rates and Likelihood of Recapture](#Survival-and-Rates-and-Likelihood-of-Recapture)
        - [Morphometrics](#Morphometrics):
            - [SVL](#SVL)
            - [TL](#TL)
            - [RTL](#RTL)
            - [Mass](#Mass)
        - [Captures](#Captures)
        - [Population Size](#Population-Size)
        - [Species Distribution](#Species-Distribution)
        - [Sex Distribution](#Sex-Distribution)
        - [Tail Condition Distribution](#Tail-Condition-Distribution)
        - [Location](#Location)
1. [Summary](#Summary)
5. [Export Files](#Export-Files)

# Things to Do
- Reorganize sections 
- Remove extraneous sections

## [Resume Here](#resume)

## Reducing the analyses sample by date range and capture

In [66]:
# convert date to pandas datetime
#df.date=pd.to_datetime(df.date)
# limiting months to between May and August
# df = df.loc[(df.date.dt.month>=5) & (df.date.dt.month<=8)]
# limit to first captures
df_lastInYear = df.loc[~df.duplicated(subset=['liznumber','year'],keep='last')]
df_firstInYear = df.loc[~df.duplicated(subset=['liznumber','year'])]

## Introduction

This notebook contains code and output of descriptive analyses for the 2000-2017 CC dataset after cleaning.

The objectives of this notebook are to describe the community of the _Sceloporus jarrovii_ and _Sceloporus virgatus_ lizards in the Crystal Creek wash from 2000 until 2017.  The population demographic metrics we examine are: [population size](#Population-Size), [sex distribution](#Sex-Distribution), [tail condition distribution](#Tail-Condition-Distribution), [location](#Location), [morphometrics](#Morphometrics) -- [SVL](#SVL), [TL](#TL), [RTL](#RTL), [mass](#Mass) --,  [survival and rates and likelihood of recapture](#Survival-and-Rates-and-Likelihood-of-Recapture), and [growth](#Growth).

We will examine these metrics and interactions among them with particular interest in the impact of environmental factors from year to year.


##  Set up Python

First we will need to set up the python environment, importing the necessary packages and setting the display options.

[Top](#TOC)

In [3]:
import pandas as pd
import numpy as np
import os, glob, logging
from summary_functions import *
from scipy import stats
from monthlit import *


import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
plotly.tools.set_config_file(world_readable=True)


# increase print limit
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 50

### Setting File Locations

In [4]:
deviceDict = {'dataBig':{'source':'S:/Chris/TailDemography/TailDemography/outputFiles'
                         ,'log':'S:/Chris/TailDemography/TailDemography/Scripts and notes/Descriptive/'
                         ,'output':'S:/Chris/TailDemography/TailDemography/outputFiles/'},
              'silverSurfer':{'source':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles'
                              ,'log':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                              ,'output':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles\\Descriptive\\'}
              ,'dataPers':{'source':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
                           ,'log': 'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                           ,'output':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles/Descriptve/'}}

### Choose Device

In [5]:
device = deviceDict['dataBig']
device

{'source': 'S:/Chris/TailDemography/TailDemography/outputFiles',
 'log': 'S:/Chris/TailDemography/TailDemography/Scripts and notes/Descriptive/',
 'output': 'S:/Chris/TailDemography/TailDemography/outputFiles/'}

# Source Data


### Logging

In [6]:
logging.basicConfig(filename=device['log']+'Desriptive Analyses.log'
                    , filemode='a',
                    format='%(funcName)s - %(levelname)s - %(message)s - %(asctime)s', level=logging.DEBUG)

## Functions

This section contains functions that were created for this notebook.

- [distribution](#distribution)
- [monthlit](#monthlit)

### distribution
[Back to Top](#TOC)

[Back to Functions](#Functions)

*distribution* takes a series or list of numeric objects, *x*, and returns descriptive stats of x including
        n, minimum, maximum, median, sIQR, mean, and stdev
    
Here are a few examples of how *distribution* works.

In [7]:
foo = [0,1,2,'r']
distribution(foo)

In [8]:
bar = [0,1,2]
distribution(bar)

,n,minimum,maximum,median,siqr,mean,stdev
0,3,0,2,1.0,0.5,1.0,1.0


[Back to Functions](#Functions)

## monthlit
[Back to Top](#TOC)

[Back to Functions](#Functions)

Here are a few examples of how _monthlit_ works.

In [9]:
dates = pd.DataFrame(data={'dates':['2018-12-9','2019-8-5', '2017/7/4',np.nan,None]})
dates.dates = pd.to_datetime(dates.dates)
dates

,dates
0,2018-12-09
1,2019-08-05
2,2017-07-04
3,NaT
4,NaT


In [10]:
np.isnan(np.nan)

True

In [11]:
monthlit(dates.dates.dt.month[0])

'Dec'

In [12]:
dates.dates.dt.month.apply(monthlit)

0    Dec
1    Aug
2    Jul
3    NaN
4    NaN
Name: dates, dtype: object

[Back to Functions](#Functions)

## Get Data
[Top](#TOC)

Here we can set the locations from which we get data and to which we export it.

We'll display all files in the source folder with the prefix _'cleaned CC data 2000-2017'_. The file names will be saved in a variable, _mysourcefiles_.

In [29]:
os.chdir(device['source'])
mysourcefiles = glob.glob('cleaned CC data 2000-2017_*.csv')
mysourcefiles

['cleaned CC data 2000-2017_2019-01-31 01hrs43min.csv',
 'cleaned CC data 2000-2017_2019-03-10 14hrs42min.csv',
 'cleaned CC data 2000-2017_2019-03-12 21hrs48min.csv',
 'cleaned CC data 2000-2017_2019-03-12 22hrs52min.csv',
 'cleaned CC data 2000-2017_2019-03-12 22hrs55min.csv',
 'cleaned CC data 2000-2017_2019-04-25 00hrs58min.csv',
 'cleaned CC data 2000-2017_2019-04-25 01hrs00min.csv',
 'cleaned CC data 2000-2017_2019-05-02 00hrs17min.csv',
 'cleaned CC data 2000-2017_2019-05-02 01hrs03min.csv',
 'cleaned CC data 2000-2017_2019-05-02 01hrs08min.csv',
 'cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv']

Automate getting the latest file

In [34]:
latestTest = [latestFile for latestFile in mysourcefiles if \
 max({pd.to_datetime(afile.split("_")[1].split(".csv")[0].split(' ')[0]) \
      for afile in mysourcefiles}) == pd.to_datetime(latestFile.split("_")\
                                                     [1].split(".csv")[0].split(' ')[0])]
latest = latestTest[0] if len(latestTest)==1 else "You must manually select a value for 'latest'."
latest

'cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv'

In [35]:
min({afile.split(' ')[-1].replace('hrs','').replace('min.csv','') for afile in mysourcefiles})

'0017'

The most recent file is the one we will use as _df_ in our descriptive analysis.

In [36]:
df=pd.read_csv(latest)
df.head()

,species,toes_orig,sex,date,svl,tl,rtl,autotomized,mass,location,meters,newRecap,painted,sighting,paint.mark,vial,misc,rtl_orig,toes,toe_pattern,description,action,pattern_b,replacement,year,tl_svl,mass_svl,all_meters,initialCaptureDate,year_diff,smallest_svl,svl_diff,liznumber,sex_count,daysSinceCapture,capture
0,j,3-7-11-19,m,2002-07-14,63.0,92.0,0.0,False,10.0,halfway up to site,-200,N,NaN,NaN,b101t,toes in vial 58-02,NaN,0.0,3-7-11-19,NaN,NaN,NaN,NaN,NaN,2002,1.460317,0.158730,['-200'],2002-07-14,0,63.0,0.0,375,1,0,1
1,j,3-7-11-18,m,2002-07-14,66.0,92.0,0.0,False,10.8,left downstream 100m v 1 falls,-100,N,NaN,NaN,b102t,toes in vial 59-02,NaN,0.0,3-7-11-18,NaN,NaN,NaN,NaN,NaN,2002,1.393939,0.163636,['-100' '87'],2002-07-14,0,66.0,0.0,374,1,0,1
2,j,3-7-12-16,m,2002-07-14,68.0,103.0,0.0,False,10.3,90m v 1 falls,-90,N,NaN,NaN,b103t,toes in vial 60-02,NaN,0.0,3-7-12-16,NaN,NaN,NaN,NaN,NaN,2002,1.514706,0.151471,['-90'],2002-07-14,0,68.0,0.0,376,1,0,1
3,j,10,m,2002-07-14,85.0,118.0,0.0,False,19.5,sb - trail intersection v 1 falls,-20,R,toe loss may be natural,NaN,b104t,NaN,NaN,0.0,10,NaN,NaN,NaN,NaN,NaN,2002,1.388235,0.229412,['-20' '-12' '-35'],2002-07-14,0,85.0,0.0,830,2,0,1
4,v,10-16,f,2002-07-03,63.0,49.0,21.0,True,11.6,left side @ base juniper 8 m ^ sb; 15m v 1 falls,-15,N,painted; gravid; <5 mites in pockets,NaN,w1a,toes in vial 34-02 (10-16),NaN,21.0,10-16,NaN,NaN,NaN,NaN,NaN,2002,0.777778,0.184127,['-15'],2002-07-03,0,63.0,0.0,1414,1,0,1


## Analyze Data
[Top](#Table-of-Contents)

We will first examine the range and distribution of number of variables in our data set:
- [Population Size](#Population-Size)
- [Species Distribution](#Species-Distribution)
- [Sex Distribution](#Sex-Distribution)
- [Tail Condition Distribution](#Tail-Condition-Distribution)
- [Location](#Location)
- [Morphometrics](#Morphometrics):
    - [SVL](#SVL)
    - [TL](#TL)
    - [RTL](#RTL)
    - [Mass](#Mass)
- [Growth](#Growth)
    - [SVL Growth](#SVL-Growth)
    - [TL Growth](#TL-Growth)
    - [RTL Growth](#RTL-Growth)
    - [Mass Growth](#Mass-Growth)
    
We will use first captures of each lizard in a year for these analyses with the exception of the growth analyses.

## Population Size

We will begin by examining the range and distribution of _species_ values.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [67]:
populationSize = df_firstInYear.groupby(['year','species']).liznumber.nunique().reset_index()
populationSize

,year,species,liznumber
0,2000.0,j,153
1,2001.0,j,135
2,2001.0,other,1
3,2002.0,j,119
4,2002.0,other,20
5,2002.0,v,79
6,2003.0,j,97
7,2003.0,other,25
8,2003.0,v,74
9,2004.0,j,70


In [68]:
Sjarrovii = go.Scatter(x = populationSize.loc[populationSize.species=='j'].year
           , y = populationSize.loc[populationSize.species=='j'].liznumber
          ,name = 'S. jarrovii')
Svirgatus = go.Scatter(x = populationSize.loc[populationSize.species=='v'].year
           , y = populationSize.loc[populationSize.species=='v'].liznumber
          ,name = 'S. virgatus')
data = [Sjarrovii, Svirgatus]
layout = go.Layout(
    title = 'Population Size for Sceloporus jarrovii and S. virgatus 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18))
)
fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Population Size for Sceloporus jarrovii and S. virgatus 2000-2017.html')

### Species Distribution

We will begin by examining the range and distribution of _species_ values.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [69]:
df_firstInYear.species.value_counts(dropna=False)

j        1471
v         709
other     212
NaN         1
Name: species, dtype: int64

These values should only be _'j', 'v', and 'other'_ any other values will be removed in the cleaning notebook and excluded from further analyses here.

In [70]:
df_firstInYear = df_firstInYear.loc[df_firstInYear.species.isin(['j','v','other'])]
df_firstInYear.species.value_counts(dropna=False)

j        1471
v         709
other     212
Name: species, dtype: int64

### Sex Distribution

We will begin by examining the range and distribution of _sex_ values.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [71]:
df_firstInYear.sex.value_counts(dropna=False)

f    1250
m    1142
Name: sex, dtype: int64

These values should only be _'m' and 'f'_.  Any other values will be removed in the cleaning notebook and excluded from further analyses here.

In [72]:
df_firstInYear = df_firstInYear.loc[df_firstInYear.sex.isin(['m','f'])]
df_firstInYear.sex.value_counts(dropna=False)

f    1250
m    1142
Name: sex, dtype: int64

<a id = 'Autotomy'></a>

### Tail Condition Distribution

Here we look at the proportion of individuals in our data who have experienced autotomy.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [73]:
#need to get rid of duplicate lizard numbers in each year (in case of a change in autotomy status)
#set bin size to 1
jarrovii_aut = go.Bar(y = df_firstInYear.loc[(df_firstInYear.species=='j')&(df_firstInYear.autotomized==True)]\
                            .groupby(['year']).liznumber.nunique()
                     ,name = 'S. jarrovii (Autotomized)',x=df_firstInYear.year.unique())
jarrovii_int = go.Histogram(y = df_firstInYear.loc[(df_firstInYear.species=='j')&(df_firstInYear.autotomized==False)]\
                            .groupby(['year']).liznumber.nunique()
                     ,name = 'S. jarrovii (Intact)',x=df_firstInYear.year.unique())
virgatus_aut = go.Histogram(y = df_firstInYear.loc[(df_firstInYear.species=='v')&(df_firstInYear.autotomized==True)]\
                            .groupby(['year']).liznumber.nunique()
                     ,name = 'S. virgatus (Autotomized)',x=df_firstInYear.year.unique())
virgatus_int = go.Histogram(y = df_firstInYear.loc[(df_firstInYear.species=='v')&(df_firstInYear.autotomized==False)]\
                            .groupby(['year']).liznumber.nunique()
                     ,name = 'S. virgatus (Intact)',x=df_firstInYear.year.unique())
data = [jarrovii_aut, jarrovii_int, virgatus_aut, virgatus_int]
layout = go.Layout(
    title = 'Distribution of Autotomy for Sceloporus jarrovii and S. virgatus 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18))
)
fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Distribution of Autotomy for Sceloporus jarrovii and S. virgatus 2000-2017')

Let's look at how this distribution changes throughout the years.

<a id = 'resume'></a>

[Back to Top](#TOC)

### Morphometrics

In this section we describe the distributions of various morphometrics.

- [SVL](#SVL)
- [TL](#TL)
- [RTL](#RTL)
- [Mass](#SVL)

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

#### SVL

Now we examine the range and distribution of svl values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of SVL](#SVLhist)

In [37]:
print("SVL values in the data set range from {} to {} for and are distributed across species and sex \
as displayed here:"\
      .format(df_firstInYear.svl.min(), df_firstInYear.svl.max()))
df_firstInYear.groupby(['species','sex']).svl.apply(distribution)

NameError: name 'df_firstInYear' is not defined

Let's plot these values. 

##### Histogram of SVL values

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)


In [75]:
femaleSj = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='j')&(df_firstInYear.sex=='f')].svl,name='Sj females')
maleSj = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='j')&(df_firstInYear.sex=='m')].svl,name='Sj males')
femaleSv = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='v')&(df_firstInYear.sex=='f')].svl,name='Sv females')
maleSv = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='v')&(df_firstInYear.sex=='m')].svl,name='Sv males')

data = [maleSj,maleSv,femaleSj,femaleSv]
layout = go.Layout(
    title = 'Histogram of SVL at Capture for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'SVL at time of capture',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of SVL at Capture for CC 2000-2017')

 Outliers will be addressed in the Cleaning notebook, but will be removed for the remained of the analyses here.

In [76]:
df_firstInYear.loc[(df_firstInYear.species=='v')&(df_firstInYear.svl==79)] = None
print("svl values in the data set range from {} to {} for and are distributed across species and sex \
as displayed here:"\
      .format(df_firstInYear.svl.min(), df_firstInYear.svl.max()))
df_firstInYear.groupby(['species','sex']).svl.apply(distribution)

svl values in the data set range from 13.0 to 112.0 for and are distributed across species and sex as displayed here:


n  minimum  maximum  median  siqr       mean      stdev
species sex                                                             
j       f   0  795     13.0     86.0    65.0   9.5  61.542138  14.155369
        m   0  676     24.0     98.0    73.0  14.0  68.008876  18.280204
other   f   0  129     32.0    105.0    48.0   2.0  49.310078   9.934786
        m   0   83     40.0    112.0    49.0   1.5  51.409639  12.600848
v       f   0  326     31.0     66.0    56.0   3.5  55.070552   5.885936
        m   0  383     28.0     60.0    51.0   3.0  50.459530   4.683468

#### TL

Now we examine the range and distribution of TL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of TL](#TLhist)

In [77]:
print("tl values among intact lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df_firstInYear.tl.min(), df_firstInYear.tl.max()))
df_firstInYear.loc[df_firstInYear.autotomized==False].groupby(['species','sex']).tl.apply(distribution)

tl values among intact lizards in the data set range from 7.0 to 165.0 for and are distributed across species and sex as displayed here:


n  minimum  maximum  median    siqr       mean      stdev
species sex                                                               
j       f   0  574     10.0    121.0    84.0  15.000  79.606272  19.699798
        m   0  462     12.0    132.0    95.0  23.375  87.729437  26.880759
other   f   0   89     37.0    129.0    67.0   4.500  67.438202  11.809044
        m   0   64     58.0    132.0    78.0   5.250  80.328125  13.935287
v       f   0  267     30.0     83.0    69.0   4.000  68.640449   7.468098
        m   0  297     30.0     86.0    69.0   5.000  67.323232   7.973556

In [78]:
print("tl values among autotomized lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df_firstInYear.tl.min(), df_firstInYear.tl.max()))
df_firstInYear.loc[df_firstInYear.autotomized==True].groupby(['species','sex']).tl.apply(distribution)

tl values among autotomized lizards in the data set range from 7.0 to 165.0 for and are distributed across species and sex as displayed here:


n  minimum  maximum  median   siqr       mean      stdev
species sex                                                              
j       f   0  221      7.0    102.0    66.0  11.00  63.914027  19.096569
        m   0  214      7.0    125.0    76.0  13.00  72.929907  24.716035
other   f   0   40     13.0    165.0    55.0   5.75  58.200000  29.722822
        m   0   19     14.0     78.0    59.0  10.50  56.368421  15.553530
v       f   0   59     10.0     63.0    49.0   5.75  46.559322  10.614355
        m   0   85     12.0     68.0    49.0   7.00  45.352941  13.916166

#### RTL

Now we examine the range and distribution of RTL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of RTL](#RTLhist)

In [79]:
print("rtl values among autotomized lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df_firstInYear.rtl.min(), df_firstInYear.rtl.max()))
df_firstInYear.loc[df_firstInYear.autotomized==True].groupby(['species','sex']).rtl.apply(distribution)

rtl values among autotomized lizards in the data set range from 0.0 to 75.0 for and are distributed across species and sex as displayed here:


n  minimum  maximum  median    siqr       mean      stdev
species sex                                                               
j       f   0  221      0.0     58.0    18.0  14.500  19.113122  15.872242
        m   0  214      0.0     60.0    18.5  15.000  19.883178  16.924866
other   f   0   40      0.0     75.0    19.0  15.125  21.725000  18.905958
        m   0   19      4.0     45.0    26.0  10.250  26.000000  13.042665
v       f   0   59      0.0     39.0    26.0  13.250  20.491525  13.369256
        m   0   85      0.0     39.0    17.0  11.000  17.670588  12.746977

## Captures

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.  We will use all captures for this

In [80]:
print("The maximum number of captures among lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df_firstInYear.groupby('liznumber').capture.max().min(), df_firstInYear.groupby('liznumber').capture.max().max()))
df_firstInYear.loc[df_firstInYear.autotomized==True].groupby(['species','sex']).capture.apply(distribution)

The maximum number of captures among lizards in the data set range from 1.0 to 18.0 for and are distributed across species and sex as displayed here:


n  minimum  maximum  median  siqr      mean     stdev
species sex                                                           
j       f   0  221      1.0     18.0     1.0  0.50  1.642534  1.490516
        m   0  214      1.0      6.0     1.0  0.50  1.420561  0.811159
other   f   0   40      1.0      3.0     1.0  0.50  1.450000  0.677476
        m   0   19      1.0      2.0     1.0  0.00  1.052632  0.229416
v       f   0   59      1.0      5.0     1.0  0.25  1.474576  0.988833
        m   0   85      1.0      7.0     1.0  0.00  1.411765  0.967251

In [64]:
Sj = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='j')].groupby('liznumber').capture.max(),name='Sj')
Sv = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='v')].groupby('liznumber').capture.max(),name='Sv')

data = [Sj,Sv]
layout = go.Layout(
    title = 'Histogram of Maximum Number of Captures for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Unique Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of Maximum Number of Captures for CC 2000-2017')

In [82]:
SjIntact = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='j')&(df_firstInYear.autotomized==False)]\
                  .groupby('liznumber').capture.max(),name='Sj-Intact')
SvIntact = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='v')&(df_firstInYear.autotomized==False)]\
                  .groupby('liznumber').capture.max(),name='Sv-Intact')
SjAut = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='j')&(df_firstInYear.autotomized==True)]\
                  .groupby('liznumber').capture.max(),name='Sj-Autotomized')
SvAut = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.species=='v')&(df_firstInYear.autotomized==True)]\
                  .groupby('liznumber').capture.max(),name='Sv-Autotomized')
data = [SjIntact,SvIntact,SjAut,SvAut]
layout = go.Layout(
    title = 'Histogram of Maximum Number of Captures for CC 2000-2017 by Autotomy Status',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Unique Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of Maximum Number of Captures for CC 2000-2017 by Autotomy Status')

C:\Users\craga_eowcrpe\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



### Number of lizards (*Sj*) by year and sex

In [65]:
df_firstInYear.groupby('year').sex.value_counts()

year    sex
2000.0  f       84
        m       69
2001.0  f       72
        m       64
2002.0  f      117
        m      101
2003.0  f      108
        m       88
2004.0  f       82
        m       68
2005.0  m       90
        f       69
2006.0  m       83
        f       65
2007.0  f       82
        m       66
2008.0  f       67
        m       57
2009.0  f       83
        m       72
2010.0  m       43
        f       40
2011.0  f       36
        m       28
2012.0  f       47
        m       35
2013.0  f       51
        m       46
2014.0  f       58
        m       50
2015.0  f       81
        m       79
2016.0  m       58
        f       49
2017.0  f       59
        m       45
Name: sex, dtype: int64

Pull out all individuals that we've recaught for Sj and writes to csv

In [85]:
multicapToes=df_firstInYear.loc[(df_firstInYear.species=='j')& (df_firstInYear.toes!="")& (df_firstInYear.toes!='NA')]\
.toes.value_counts()[df_firstInYear.loc[df_firstInYear.species=='j']\
                     .toes.value_counts()>1].index.tolist()
df_firstInYear.loc[df_firstInYear.toes.isin(multicapToes)].sort_values(by=['toes','date']).shape[0]
# df_firstInYear.loc[df_firstInYear.toes.isin(multicapToes)].sort_values(by=['toes','date']).to_csv('multicaps.csv')
# multicapToes

760

### Maximum Number of Captures based on Tail condition
 - match for species, sex, size and location

In [91]:
#separate species
dfF = df_firstInYear.loc[df_firstInYear.sex =='f']
dfM = df_firstInYear.loc[df_firstInYear.sex =='m']

females = go.Histogram(x = dfF.groupby('liznumber').capture.max(),name='females')
males = go.Histogram(x = dfM.groupby('liznumber').capture.max(),name='males')

data = [males,females]
layout = go.Layout(
    title = 'Maximum Number of Captures per Individual 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of Maximum Captures per Individual in Crystal Creek 2000 - 2017')

C:\Users\craga_eowcrpe\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [94]:
SjFemales = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex =='f')&(df_firstInYear.species=='j')]\
                       .groupby('liznumber').capture.max(),name='Sj-females')
SjMales = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex =='m')&(df_firstInYear.species=='j')]\
                     .groupby('liznumber').capture.max(),name='Sj-males')
SvFemales = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex =='f')&(df_firstInYear.species=='v')]\
                       .groupby('liznumber').capture.max(),name='Sv-females')
SvMales = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex =='m')&(df_firstInYear.species=='v')]\
                     .groupby('liznumber').capture.max(),name='Sv-males')

data = [SjMales,SjFemales,SvMales,SvFemales]
layout = go.Layout(
    title = 'Maximum Number of Captures per Individual 2000-2017 by sex and species',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of Maximum Captures per Individual in Crystal Creek 2000 - 2017 by sex and species')

## Growth

- [SVL Growth](#SVL-Growth)
- [TL Growth](#TL-Growth)
- [RTL Growth](#RTL-Growth)
- [Mass Growth](#Mass-Growth)

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.

### SVL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the body size growth rate?

### TL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the tail size growth rate?

### RTL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the regrown tail size growth rate?

### Mass Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the body size growth rate in terms of MAss?

In [ ]:
df.capture.value_counts()

## Export Files
[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)